<a href="https://colab.research.google.com/github/jakkaphat1/Project1_doc/blob/main/(pythainlp%2Bn_gram%2BLaBse%2BCLIPs%2Bmatching_count%2BOpenCV)_Assignment_Plagiarism_Detection_V1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Document**
Text Embedding Model : LaBse

Lexical Matching Process : N-gram + Jaccard Similarity

--------------------------------------------
Image Embedding Model : clip-vit-large-patch14

Embedding similarity choosen : Matching Count (ใช้จำนวนภาพที่มี Similarity > threshold ที่ตั้งไว้)



# Requirements

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

import os
from slugify import slugify

import shutil


In [2]:
!pip install -q pythainlp
from pythainlp.tokenize import word_tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 41.5 MB/s eta 0:00:00


In [3]:
!pip install pymupdf
!pip install pytesseract
!pip install pillow pdf2image
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-tha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 41.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.9 [186 kB]
Fetched 186 kB in 8s (24.6 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.9_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.9) ...
Setting up poppler-utils (22.02.0-2ubuntu0.9) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-oc

In [4]:
!pip install pinecone-client transformers pymupdf


In [5]:
pip install python-slugify

In [6]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [7]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [8]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [9]:
from google.colab import files

In [10]:
import tempfile

# Embedding Models

In [11]:
import fitz  # PyMuPDF
import tempfile
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer

# โหลดโมเดล Embedding
labse_model = SentenceTransformer("sentence-transformers/LaBSE")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Pinecone

In [12]:
from pinecone import Pinecone, ServerlessSpec
# Pinecone API Key และ Environment (Region)
PINECONE_API_KEY = "pcsk_3q7EHL_2jLmXAfQ9rXqj3h2tHaG1YjjYuQ5HnEtNHucWasCCsxdvr8qvLHqPip4Aa17ziJ"
PINECONE_ENVIRONMENT = "us-east-1"  # Region

# สร้าง Pinecone Client Instance
pc = Pinecone(api_key=PINECONE_API_KEY)

# ตรวจสอบ Index ก่อนสร้าง
if "text-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

if "image-index" in pc.list_indexes().names():
    print(pc.list_indexes().names())

# โหลด Index
text_index = pc.Index("text-index")
image_index = pc.Index("image-index")



['text-1024', 'text-index', 'text-769', 'image-1024', 'image-index']
['text-1024', 'text-index', 'text-769', 'image-1024', 'image-index']


# Plagiarism Process

In [46]:
from google.colab import files

def upload_pdfs():
    uploaded = files.upload()
    pdf_paths = list(uploaded.keys())
    return pdf_paths

pdf_paths = upload_pdfs()

Saving 653380191-4.pdf to 653380191-4.pdf
Saving 653380191-2.pdf to 653380191-2.pdf
Saving 653380191-3.pdf to 653380191-3.pdf
Saving 653380191-1.pdf to 653380191-1.pdf


In [47]:
def extract_text_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    tokens = word_tokenize(text, engine="newmm")
    tokenized_text = " ".join(tokens)
    text_embedding = labse_model.encode(tokenized_text)
    return text.strip(), text_embedding

def extract_images_and_embed(pdf_path):
    doc = fitz.open(pdf_path)
    image_embeddings = []
    temp_dir = tempfile.mkdtemp()
    for page_num in range(len(doc)):
        for img_index, img_info in enumerate(doc[page_num].get_images(full=True)):
            xref = img_info[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            img_path = os.path.join(temp_dir, f"page_{page_num}_img_{img_index}.png")
            with open(img_path, "wb") as f:
                f.write(img_bytes)
            img = Image.open(img_path).convert("RGB")
            inputs = clip_processor(images=img, return_tensors="pt")
            with torch.no_grad():
                img_embedding = clip_model.get_image_features(**inputs)
            image_embeddings.append(img_embedding.squeeze().numpy())
    shutil.rmtree(temp_dir)
    return image_embeddings


In [48]:
def to_ascii_id(filename):
    return slugify(filename, separator="_")

In [49]:
def upload_text_embedding(doc_id, text_embedding):
    if text_embedding is not None and len(text_embedding) > 0:
        text_index.upsert([(f"text_{doc_id}", text_embedding.tolist(), {"type": "text"})])

def upload_image_embedding(doc_id, image_embeddings):
    if image_embeddings:
        for i, img_emb in enumerate(image_embeddings):
            image_index.upsert([(f"image_{doc_id}_{i}", img_emb.tolist(), {"type": "image"})])

In [50]:
raw_texts = {}

for pdf_path in pdf_paths:
    doc_id = to_ascii_id(os.path.basename(pdf_path).split('.')[0])
    text, text_embedding = extract_text_and_embed(pdf_path)
    image_embeddings = extract_images_and_embed(pdf_path)
    upload_text_embedding(doc_id, text_embedding)
    upload_image_embedding(doc_id, image_embeddings)
    raw_texts[doc_id] = text

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [51]:
# def compare_images(image_embeddings, doc_id_1, doc_id_2):
#     max_similarity = 0.0
#     keys1 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_1}_")]
#     keys2 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_2}_")]
#     for k1 in keys1:
#         for k2 in keys2:
#             sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
#             max_similarity = max(max_similarity, sim)
#     return max_similarity

# def top_k_image_similarity(image_embeddings, doc_id_1, doc_id_2, k=3):
#     similarities = []
#     keys1 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_1}_")]
#     keys2 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_2}_")]

#     for k1 in keys1:
#         for k2 in keys2:
#             sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
#             # print(f"[DEBUG] Comparing {k1} with {k2}: Similarity = {sim:.4f}")
#             similarities.append(sim)

#     if not similarities:
#         return 0.0

#     similarities.sort(reverse=True)
#     top_k = similarities[:min(k, len(similarities))]
#     return sum(top_k) / len(top_k)

def matching_count(image_embeddings, doc_id_1, doc_id_2, threshold=0.8):
    similarities = []
    keys1 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_1}_")]
    keys2 = [k for k in image_embeddings if k.startswith(f"image_{doc_id_2}_")]

    for k1 in keys1:
        for k2 in keys2:
            sim = cosine_similarity([image_embeddings[k1]], [image_embeddings[k2]])[0][0]
            similarities.append(sim)

    # นับจำนวนคู่ที่ similarity มากกว่า threshold
    matched = [sim for sim in similarities if sim >= threshold]

    # คำนวณ Matching Ratio: จำนวนคู่ที่ตรงเกณฑ์ / จำนวนคู่ทั้งหมด
    matching_ratio = len(matched) / len(similarities) if similarities else 0.0

    return matching_ratio



In [52]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = word_tokenize(text, engine="newmm")
    return " ".join(tokens)

def compute_lexical_similarity(text1, text2, ngram_range=(1,3)):
    t1, t2 = preprocess_text(text1), preprocess_text(text2)
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=ngram_range)
    tfidf = vectorizer.fit_transform([t1, t2])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

def compute_hybrid_similarity(text1, text2, emb1, emb2, alpha=0.5, beta=0.5):
    semantic = cosine_similarity([emb1], [emb2])[0][0]
    lexical = compute_lexical_similarity(text1, text2)
    hybrid = alpha * semantic + beta * lexical
    return hybrid, semantic, lexical


In [53]:
def get_all_text_embeddings():
    stats = text_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]
    if total_vectors == 0:
        return {}
    text_vectors = text_index.query(vector=[0]*labse_model.get_sentence_embedding_dimension(), top_k=total_vectors, include_values=True)
    return {match["id"]: np.array(match["values"]) for match in text_vectors["matches"]}

def get_all_image_embeddings():
    stats = image_index.describe_index_stats()
    total_vectors = stats["total_vector_count"]

    if total_vectors == 0:
        print("ไม่มีรูปภาพใน index")
        return {}

    #ใช้ query() เพื่อดึงค่าเวกเตอร์ทั้งหมดแทน fetch()
    image_vectors = image_index.query(
        vector=[0] * 768,
        top_k=total_vectors,
        include_values=True
    )

    return {match["id"]: np.array(match["values"]) for match in image_vectors["matches"]}


In [74]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# def compare_all_documents():
#     text_embeddings = get_all_text_embeddings()
#     image_embeddings = get_all_image_embeddings()
#     results = []

#     doc_ids = list(raw_texts.keys())
#     for i in range(len(doc_ids)):
#         for j in range(i + 1, len(doc_ids)):
#             id1, id2 = doc_ids[i], doc_ids[j]
#             t1, t2 = raw_texts[id1], raw_texts[id2]
#             e1, e2 = text_embeddings[f"text_{id1}"], text_embeddings[f"text_{id2}"]

#             hybrid, semantic, lexical = compute_hybrid_similarity(t1, t2, e1, e2)
#             # image_sim = compare_images(image_embeddings, id1, id2)
#             image_sim = top_k_image_similarity(image_embeddings, id1, id2, k=3)

#             level = "เหมือนมาก" if hybrid >= 0.9 else \
#                     "คล้ายสูง" if hybrid >= 0.75 else \
#                     "คล้ายระดับกลาง" if hybrid >= 0.6 else "ไม่คล้าย"

#             print(f" {id1} vs {id2} [{level}]")
#             print(f"  → Semantic: {semantic:.4f} | Lexical: {lexical:.4f} | Hybrid: {hybrid:.4f} | Image: {image_sim:.4f}")
#             results.append({
#                 "doc_1": id1,
#                 "doc_2": id2,
#                 "hybrid_similarity": hybrid,
#                 "semantic_similarity": semantic,
#                 "lexical_similarity": lexical,
#                 "image_similarity": image_sim,
#                 "level": level
#             })
#     return results

def compare_all_documents():
    text_embeddings = get_all_text_embeddings()
    image_embeddings = get_all_image_embeddings()
    results = []

    doc_ids = list(raw_texts.keys())
    for i in range(len(doc_ids)):
        for j in range(i + 1, len(doc_ids)):
            id1, id2 = doc_ids[i], doc_ids[j]
            t1, t2 = raw_texts[id1], raw_texts[id2]
            e1, e2 = text_embeddings[f"text_{id1}"], text_embeddings[f"text_{id2}"]

            hybrid, semantic, lexical = compute_hybrid_similarity(t1, t2, e1, e2)

            # ใช้ Matching Count แทน Top-K
            image_sim = matching_count(image_embeddings, id1, id2, threshold=0.8)

            # ข้อความเกี่ยวกับการคล้ายคลึงของ Text
            if semantic < 0.7:
                text_message = "ข้อความมีความคล้ายน้อย"
            else:
                text_message = "ข้อความมีความคล้ายคลึงสูง"

            # แสดงผล Image: เปลี่ยนเป็นข้อมูลที่แสดงว่าภาพเหมือนกี่ในกี่
            keys1 = [k for k in image_embeddings if k.startswith(f"image_{id1}_")]
            keys2 = [k for k in image_embeddings if k.startswith(f"image_{id2}_")]
            total_images = len(keys1) * len(keys2)
            matched_images = int(image_sim * total_images)  # หาคู่ที่ตรงตามเกณฑ์

            # แสดงผลลัพธ์
            level = "เหมือนมาก" if hybrid >= 0.9 else \
                    "คล้ายสูง" if hybrid >= 0.75 else \
                    "คล้ายระดับกลาง" if hybrid >= 0.6 else "ไม่คล้าย"

            print("===============================================================================")
            print(f"{id1} vs {id2} [{level}]")
            print(f"  → Semantic: {semantic:.4f} | Lexical: {lexical:.4f} | Hybrid: {hybrid:.4f} | Image: {image_sim:.4f}")
            print(f"  → Text Message: {text_message}")
            print(f"  → Image: ภาพเหมือน {matched_images} จาก {total_images} รูป")

            results.append({
                "doc_1": id1,
                "doc_2": id2,
                "hybrid_similarity": hybrid,
                "semantic_similarity": semantic,
                "lexical_similarity": lexical,
                "image_similarity": image_sim,
                "level": level,
                "text_message": text_message,
                "matched_images": matched_images,
                "total_images": total_images
            })
    return results



In [75]:
compare_results = compare_all_documents()

653380191_4 vs 653380191_2 [ไม่คล้าย]
  → Semantic: 0.4011 | Lexical: 0.0000 | Hybrid: 0.2005 | Image: 0.5000
  → Text Message: ข้อความมีความคล้ายน้อย
  → Image: ภาพเหมือน 1 จาก 2 รูป
653380191_4 vs 653380191_3 [ไม่คล้าย]
  → Semantic: 0.4011 | Lexical: 0.0000 | Hybrid: 0.2005 | Image: 0.5000
  → Text Message: ข้อความมีความคล้ายน้อย
  → Image: ภาพเหมือน 1 จาก 2 รูป
653380191_4 vs 653380191_1 [ไม่คล้าย]
  → Semantic: 0.4011 | Lexical: 0.0000 | Hybrid: 0.2005 | Image: 0.5000
  → Text Message: ข้อความมีความคล้ายน้อย
  → Image: ภาพเหมือน 2 จาก 4 รูป
653380191_2 vs 653380191_3 [เหมือนมาก]
  → Semantic: 1.0000 | Lexical: 1.0000 | Hybrid: 1.0000 | Image: 1.0000
  → Text Message: ข้อความมีความคล้ายคลึงสูง
  → Image: ภาพเหมือน 1 จาก 1 รูป
653380191_2 vs 653380191_1 [เหมือนมาก]
  → Semantic: 1.0000 | Lexical: 1.0000 | Hybrid: 1.0000 | Image: 0.5000
  → Text Message: ข้อความมีความคล้ายคลึงสูง
  → Image: ภาพเหมือน 1 จาก 2 รูป
653380191_3 vs 653380191_1 [เหมือนมาก]
  → Semantic: 1.0000 | Lexical: 1

# Controller


In [44]:
text_index.delete(delete_all=True)
image_index.delete(delete_all=True)

print("ลบข้อมูลใน pinecone ทั้งหมด")


ลบข้อมูลใน pinecone ทั้งหมด


In [45]:
import os
import glob

# ค้นหาไฟล์ .pdf ทั้งหมดใน /content
pdf_files = glob.glob('/content/*.pdf')

# ลบไฟล์ .pdf ทีละไฟล์
for file_path in pdf_files:
    try:
        os.remove(file_path)
        print(f'ลบไฟล์: {file_path}')
    except Exception as e:
        print(f'ไม่สามารถลบ {file_path} ได้: {e}')


ลบไฟล์: /content/653380191-2.pdf
ลบไฟล์: /content/653380191-1.pdf
ลบไฟล์: /content/653380191-3.pdf
